In [215]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 옵션값 설정
opt = webdriver.ChromeOptions()
opt.add_argument('headless')
opt.add_argument("User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")

# 웹드라이버를 통해 네이버 금융 ETF 페이지에 접속
drv = webdriver.Chrome('C:\\Users\\JAEHO\\Desktop\\python workspace\\chromedriver.exe', options=opt)
drv.implicitly_wait(3) #-웹페이지 전체가 넘어올때까지 기다리기
drv.get('https://finance.naver.com/sise/etf.nhn')

# 뷰티풀 수프로 테이블을 스크래핑
soup = BeautifulSoup(drv.page_source, 'lxml')
drv.quit()
table = soup.find_all("table", class_="type_1 type_etf") 
    #-파이썬에서 class는 예약어로 되어 있으니 class속성을 검색하려면 class_로 검색해야 한다.

df = pd.read_html(str(table), header=0)[0]

# 불필요한 열과 행을 삭제하고 인덱스를 재설정해서 출력
df = df.drop(columns=['Unnamed: 9'])
df = df.dropna()
df.index = range(1, len(df)+1)
# print(df)

# 링크 주소에 포함된 종목코드를 추출하여 전체 종목코드와 종목명 출력
etf_td = soup.find_all("td", class_="ctg")
etfs = {}
for td in etf_td:
    s = str(td.a["href"]).split('=') 
        #-td 태그 속 a태그에서 href 요소에 대응하는 값을 =로 분할하여 리스트에 넣음
    code = s[-1]
    etfs[td.a.text] = code
        #-etf란 딕셔너리에 key,value형식으로 저장

series=pd.Series(etfs)
print(series)

KODEX 200                069500
KODEX 단기채권               153130
KODEX 200선물인버스2X         252670
TIGER 200                102110
KODEX 삼성그룹               102780
                          ...  
KBSTAR 미국장기국채선물인버스2..    267500
KBSTAR 200생활소비재          287330
KBSTAR 팔라듐선물인버스(H)       334700
KBSTAR 200경기소비재          287310
KBSTAR 200산업재            287320
Length: 487, dtype: object


In [216]:
# series=series.add_prefix('A')

add_prefix 인덱스 앞에 문자넣어주기(add_suffix는 뒤)

In [217]:
df.set_index('종목명',inplace=True)

In [218]:
df_series=series.to_frame(name='종목코드')
df_series

,종목코드
KODEX 200,069500
KODEX 단기채권,153130
KODEX 200선물인버스2X,252670
TIGER 200,102110
KODEX 삼성그룹,102780
...,...
KBSTAR 미국장기국채선물인버스2..,267500
KBSTAR 200생활소비재,287330
KBSTAR 팔라듐선물인버스(H),334700
KBSTAR 200경기소비재,287310


In [219]:
type(df_series)

pandas.core.frame.DataFrame

to_frame 은 series의 형식은 그대로 두되 dataframe으로만 바꿔줌 (name이 column역할)

In [220]:
df_merge=pd.merge(df,df_series,left_index=True,right_index=True)
df_merge

,현재가,전일비,등락률,NAV,3개월수익률,거래량,거래대금(백만),시가총액(억),종목코드
ARIRANG 200,43290.0,80.0,+0.19%,43353.0,-0.70%,50846.0,2208.0,8918.0,152100
ARIRANG 200동일가중,12185.0,70.0,+0.58%,12181.0,+3.55%,241.0,2.0,49.0,295820
ARIRANG 200선물레버리지,50650.0,210.0,+0.42%,50676.0,-1.84%,620.0,31.0,152.0,253150
ARIRANG 200선물인버스2X,3880.0,0.0,0.00%,3876.0,-0.39%,27902.0,107.0,151.0,253160
ARIRANG ESG우수기업,9930.0,40.0,+0.40%,9939.0,+4.05%,4750.0,47.0,50.0,278420
...,...,...,...,...,...,...,...,...,...
파워 200,43880.0,40.0,+0.09%,43955.0,-0.48%,38.0,1.0,285.0,152870
파워 고배당저변동성,31820.0,10.0,+0.03%,31945.0,+5.77%,111.0,3.0,76.0,192720
파워 중기국고채,100395.0,0.0,0.00%,100417.0,-0.03%,0.0,0.0,60.0,176710
파워 코스피100,33010.0,5.0,-0.02%,33023.0,-0.92%,90.0,2.0,145.0,140950


merge 할때 각자 index와 column값이 같다면 더하기 수월해짐 
인덱스가 같으면 left_index right_index로 column값이 같다면 on=column으로 하면된다.

In [221]:
df_merge['수정종목코드']='A'+df_merge['종목코드'].map(str)
df_merge.head()

,현재가,전일비,등락률,NAV,3개월수익률,거래량,거래대금(백만),시가총액(억),종목코드,수정종목코드
ARIRANG 200,43290.0,80.0,+0.19%,43353.0,-0.70%,50846.0,2208.0,8918.0,152100,A152100
ARIRANG 200동일가중,12185.0,70.0,+0.58%,12181.0,+3.55%,241.0,2.0,49.0,295820,A295820
ARIRANG 200선물레버리지,50650.0,210.0,+0.42%,50676.0,-1.84%,620.0,31.0,152.0,253150,A253150
ARIRANG 200선물인버스2X,3880.0,0.0,0.00%,3876.0,-0.39%,27902.0,107.0,151.0,253160,A253160
ARIRANG ESG우수기업,9930.0,40.0,+0.40%,9939.0,+4.05%,4750.0,47.0,50.0,278420,A278420


df_merge.map(str) : df_merge를 str형태로 다 바꿔줘라

In [222]:
#  - .Series.unique() 를 이용한 유일한 값 찾기
#  - .value_counts() 를 이용한 유일한 값별 개수 세기
# series.value_counts()
# series.unique()



In [223]:
# del df_merge['종목코드'] #=drop([종목코드],axis=1)
# print(df_merge)

In [224]:
#거래량 순으로 배열
df_merge.head()

,현재가,전일비,등락률,NAV,3개월수익률,거래량,거래대금(백만),시가총액(억),종목코드,수정종목코드
ARIRANG 200,43290.0,80.0,+0.19%,43353.0,-0.70%,50846.0,2208.0,8918.0,152100,A152100
ARIRANG 200동일가중,12185.0,70.0,+0.58%,12181.0,+3.55%,241.0,2.0,49.0,295820,A295820
ARIRANG 200선물레버리지,50650.0,210.0,+0.42%,50676.0,-1.84%,620.0,31.0,152.0,253150,A253150
ARIRANG 200선물인버스2X,3880.0,0.0,0.00%,3876.0,-0.39%,27902.0,107.0,151.0,253160,A253160
ARIRANG ESG우수기업,9930.0,40.0,+0.40%,9939.0,+4.05%,4750.0,47.0,50.0,278420,A278420


In [225]:
print(type(df_merge['전일비'])) #dataframe은 series형들의 집합
a=list('1234')
print(a+['d']) #리스트는 리스트끼리만 추가만 가능(요소끼리 연산은 for 구문이용)

<class 'pandas.core.series.Series'>
['1', '2', '3', '4', 'd']


In [226]:
df_merge.loc['KINDEX 배당성장','등락률'][0]

'-'

In [227]:
indexes=df_merge.index
for index in indexes:
    # df_merge['전일비'][index]=df_merge['등락률'][index][0]+df_merge['전일비'].map(str)[index]
    df_merge.loc[index,'전일비']=df_merge.loc[index,'등락률'][0]+df_merge['전일비'].map(str)[index]

인덱싱구조: df_merge['전일비']=df_merge['전일비'][:]이다

주석처리한 줄은 chained indexing으로 예:df['a']['b'] df에서 a찾고 return b찾고 return하는 방식으로 비효율적이므로 한꺼번에 인덱싱하는 loc,iloc indexing을 쓰는게 좋다.

In [228]:
df_merge

,현재가,전일비,등락률,NAV,3개월수익률,거래량,거래대금(백만),시가총액(억),종목코드,수정종목코드
ARIRANG 200,43290.0,+80.0,+0.19%,43353.0,-0.70%,50846.0,2208.0,8918.0,152100,A152100
ARIRANG 200동일가중,12185.0,+70.0,+0.58%,12181.0,+3.55%,241.0,2.0,49.0,295820,A295820
ARIRANG 200선물레버리지,50650.0,+210.0,+0.42%,50676.0,-1.84%,620.0,31.0,152.0,253150,A253150
ARIRANG 200선물인버스2X,3880.0,00.0,0.00%,3876.0,-0.39%,27902.0,107.0,151.0,253160,A253160
ARIRANG ESG우수기업,9930.0,+40.0,+0.40%,9939.0,+4.05%,4750.0,47.0,50.0,278420,A278420
...,...,...,...,...,...,...,...,...,...,...
파워 200,43880.0,+40.0,+0.09%,43955.0,-0.48%,38.0,1.0,285.0,152870,A152870
파워 고배당저변동성,31820.0,+10.0,+0.03%,31945.0,+5.77%,111.0,3.0,76.0,192720,A192720
파워 중기국고채,100395.0,00.0,0.00%,100417.0,-0.03%,0.0,0.0,60.0,176710,A176710
파워 코스피100,33010.0,-5.0,-0.02%,33023.0,-0.92%,90.0,2.0,145.0,140950,A140950


In [239]:
type(df_merge['거래량순위']>10)

pandas.core.series.Series

In [240]:
# df_merge.sort_values(by='거래량',ascending=False)
df_merge['거래량순위']=df_merge['거래량'].rank(ascending=False)

In [249]:
df_volume=df_merge[df_merge['거래량순위']<10]
df_volume.sort_values(by='거래량순위',ascending=True) #ascending='True' 작은게 위로

,현재가,전일비,등락률,NAV,3개월수익률,거래량,거래대금(백만),시가총액(억),종목코드,수정종목코드,거래량순위
KODEX 200선물인버스2X,1940.0,-5.0,-0.26%,1937.0,0.00%,123434581.0,237089.0,22079.0,252670,A252670,1.0
KODEX 코스닥150선물인버스,4310.0,+15.0,+0.35%,4317.0,-1.49%,21516543.0,92252.0,4913.0,251340,A251340,2.0
KODEX 인버스,3770.0,-5.0,-0.13%,3772.0,0.00%,16395709.0,61599.0,10711.0,114800,A114800,3.0
KODEX 레버리지,27865.0,+100.0,+0.36%,28082.0,-2.22%,11490447.0,322856.0,14657.0,122630,A122630,4.0
KODEX 코스닥150 레버리지,15865.0,-115.0,-0.72%,15993.0,-0.59%,6758066.0,108138.0,9154.0,233740,A233740,5.0
TIGER 200선물인버스2X,2010.0,-10.0,-0.50%,2011.0,+0.25%,3463893.0,6918.0,1146.0,252710,A252710,6.0
KODEX 200,42875.0,+70.0,+0.16%,42935.0,-0.79%,2567440.0,110490.0,39981.0,069500,A069500,7.0
KODEX 2차전지산업,20950.0,+105.0,+0.50%,21001.0,+6.03%,1423451.0,29820.0,9323.0,305720,A305720,8.0
KBSTAR 단기국공채액티브,102100.0,00.0,0.00%,102100.0,+0.08%,1142647.0,116668.0,2786.0,272560,A272560,9.0
